## Version Propia

In [99]:
from spire.doc import *
from spire.doc.common import *
from langchain import PromptTemplate
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
from io import StringIO
import os

dir_files = os.listdir(".")

docx_files = [f for f in dir_files if f.endswith(".docx")]

for file in docx_files:
    # Create a Document object
    document_obj = Document()

    # Load a Word document
    document_obj.LoadFromFile(file)

    # Extract the text of the document
    document_text = document_obj.GetText()

    # Write the extracted text into a text file
    with open("DocumentText.txt", "w", encoding="utf-8") as file_obj:
        file_obj.write(document_text)

    document_obj.Close()

    with open("DocumentText.txt", "r", encoding="utf-8") as file_obj:
        document = file_obj.read()

    template = """
    - Role: Actúa como un asistente especialista en extrar datos de documentos de texto y transformarlos a formato de tabla.
    - Instructions: Revisa el documento entregado, analiza su estructura y crea un tabla para visualizarlo de mejor manera. Entrega solo la tabla, no agregues comentarios adicionales. La tabla debe tener las siguientes columnas: "Mes", "Día", "Asignatura", "Evaluación", "Ponderación", "Tipo" y "Profesor".
    - Context: El documento contiene información sobre un cronograma de evaluaciones.
    - Objective: Extraer el texto desde el documento y transformarlo a formato de tabla.
    - Needed Data:
    * Documento de Texto
    - Tasks:
    1. Analizar el documento entregado
    2. Comprender su estructura
    3. Crear un tabla con todos los datos del documento organizados en columnas: "Mes", "Día", "Asignatura", "Evaluación", "Ponderación", "Tipo" y "Profesor".
    # Documento de Texto
    {document}
    """
    prompt_template = PromptTemplate(input_varables=["document"], template=template)

    prompt_value = prompt_template.format(document=document)

    load_dotenv()
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=OPENAI_API_KEY)

    content = "Actúa como un asistente experto que permite extraer datos de un documento de texto y transformalos a formato tabular"

    response = client.chat.completions.create(
        model="gpt-4o",  # Se puede cambiar el modelo
        messages=[
            {"role": "assistant", "content": content},
            {"role": "user", "content": prompt_value},
        ],
        max_tokens=3000,
    )

    response_content = response.choices[0].message.content

    cols = ["Mes", "Día", "Asignatura", "Evaluación", "Ponderación", "Tipo", "Profesor"]

    # Remove leading and trailing whitespace and newlines
    data = response_content.strip()

    # Use StringIO to simulate a file object
    data_io = StringIO(response_content)

    # Read the data into a Pandas DataFrame
    df = pd.read_csv(data_io, sep="|", skipinitialspace=True, skiprows=2, header=None, on_bad_lines='skip')

    # Drop the first and last empty columns created by the separator
    df = df.iloc[:, 1:-1]

    col_names = {key: value for key, value in zip(list(df.columns), cols)}

    df.rename(columns=col_names, inplace=True)

    doc = file.split()[:2]
    doc_name = " ".join(doc)

    df.to_excel(f"excel_files/{doc_name}.xlsx", sheet_name=doc_name)


## Version Large 2 Mistral AI

In [100]:
from spire.doc import *
from spire.doc.common import *
from langchain import PromptTemplate
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
from io import StringIO
import os
# from openpyxl import load_workbook

# List all files in the current directory
dir_files = os.listdir(".")

# Filter out .docx files
docx_files = [f for f in dir_files if f.endswith(".docx")]

# Create an Excel writer object
excel_writer = pd.ExcelWriter("output.xlsx", engine='openpyxl')

for file in docx_files:
    # Create a Document object
    document_obj = Document()

    # Load a Word document
    document_obj.LoadFromFile(file)

    # Extract the text of the document
    document_text = document_obj.GetText()

    # Write the extracted text into a text file
    with open("DocumentText.txt", "w", encoding="utf-8") as file_obj:
        file_obj.write(document_text)

    document_obj.Close()

    with open("DocumentText.txt", "r", encoding="utf-8") as file_obj:
        document = file_obj.read()

    template = """
    - Role: Actúa como un asistente especialista en extrar datos de documentos de texto y transformarlos a formato de tabla.
    - Instructions: Revisa el documento entregado, analiza su estructura y crea un tabla para visualizarlo de mejor manera. Entrega solo la tabla, no agregues comentarios adicionales. La tabla debe tener las siguientes columnas: "Mes", "Día", "Asignatura", "Evaluación", "Ponderación", "Tipo" y "Profesor".
    - Context: El documento contiene información sobre un cronograma de evaluaciones.
    - Objective: Extraer el texto desde el documento y transformarlo a formato de tabla.
    - Needed Data:
    * Documento de Texto
    - Tasks:
    1. Analizar el documento entregado
    2. Comprender su estructura
    3. Crear un tabla con todos los datos del documento organizados en columnas: "Mes", "Día", "Asignatura", "Evaluación", "Ponderación", "Tipo" y "Profesor".
    # Documento de Texto
    {document}
    """
    prompt_template = PromptTemplate(input_variables=["document"], template=template)

    prompt_value = prompt_template.format(document=document)

    load_dotenv()
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=OPENAI_API_KEY)

    content = "Actúa como un asistente experto que permite extraer datos de un documento de texto y transformalos a formato tabular"

    response = client.chat.completions.create(
        model="gpt-4o",  # Se puede cambiar el modelo
        messages=[
            {"role": "assistant", "content": content},
            {"role": "user", "content": prompt_value},
        ],
        max_tokens=3000,
    )

    response_content = response.choices[0].message.content

    cols = ["Mes", "Día", "Asignatura", "Evaluación", "Ponderación", "Tipo", "Profesor"]

    # Remove leading and trailing whitespace and newlines
    data = response_content.strip()

    # Use StringIO to simulate a file object
    data_io = StringIO(response_content)

    # Read the data into a Pandas DataFrame
    df = pd.read_csv(data_io, sep="|", skipinitialspace=True, skiprows=2, header=None)

    # Drop the first and last empty columns created by the separator
    df = df.iloc[:, 1:-1]

    col_names = {key: value for key, value in zip(list(df.columns), cols)}

    df.rename(columns=col_names, inplace=True)

    # Use the original file name to create the sheet name
    doc_name = " ".join(file.split()[:2])

    # Save the DataFrame to the Excel file
    df.to_excel(excel_writer, sheet_name=doc_name, index=False)

# Save the Excel file
excel_writer.save()

AttributeError: 'OpenpyxlWriter' object has no attribute 'save'